# My hangman file

In [2]:
import json
import numpy as np
import math
import requests
import random
import string
import secrets
import time
import re
import collections
from nltk import ngrams, FreqDist

In [44]:
class MyHangman(object):
    def __init__(self, weight_length = 1, weight_probability = 0.5, weight_count = 0.5, correct_word = "", full_dictionary_location = "words_250000_train.txt"):
        self.weight_length = weight_length
        self.weight_probability = weight_probability
        self.weight_count = weight_count
        self.guessed_letters = []
        if correct_word != "" :
            self.givenExternalWord = True
        else:
            self.givenExternalWord = False
        self.word = ""
        self.correct_word = correct_word
        full_dictionary_location = full_dictionary_location
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        self.current_dictionary = []
        self.alphabet = list(string.ascii_lowercase)
        self.vowels = list(["a", "e", "i", "o", "u"])
        self.guess_list = []
        self.has_won = False
        self.gameLog = []
        #self.init_grams()
        self.allNGrams=self.generateNGrams(self.full_dictionary, 5)


    def changeWeights(self, wl, wp, wc):
        self.weight_length = wl
        self.weight_probabolity = wp
        self.weight_count = wc


    def generateNGrams(self, vocab, n):
        #return the ngrams as dictionaries
        #I used the freq_dists and ngrams from nltk but they are way too slow
        #this is a little confusing because I am loading words from a dictionary (not the data type) of words...
        #let's call it vocab instead
        allNGrams = []
        for k in range(2, n+1):
            ngrams = {}
            for word in vocab:
                word = "." + word + "." # pad with start and end bits
                if len(word) < k:
                    continue
                for i in range(len(word) - k + 1):
                    gram = word[i:i+k]
                    if gram not in ngrams:
                        ngrams[gram] = 1
                    else:
                        ngrams[gram] += 1
            allNGrams.append(ngrams.copy())

        return allNGrams
        
        
    def generate_correct_word(self):
        #Ok, so by default the code will generate a random correct word from the full dictionary
        #You can also initialize the class with an external correct word, or set one after initialization

        if self.givenExternalWord == False:
            self.correct_word = random.choice(self.full_dictionary)
    
    def set_correct_word(self, givenWord):
        self.correct_word = givenWord
        self.givenExternalWord = True


    def find_potential_guesses(self, allGrams, input_word, guess_list):
        
        potential_guesses = []
        remove_set = set(guess_list)
        remove_set.add(None)

        substr_list = []
        input_word = "." + input_word + "." #pad beginning and end with something to indicate beginning/end

        for i,char in enumerate(input_word):
            if char == "_": #found a blank
                start, end = i, i
                #print(i)
                #get longest substring containing the blank
                while start > 0 and input_word[start-1] != "_":
                    start-=1
                    #print(start)
                while end < len(input_word) - 1 and input_word[end+1] != "_":
                    end += 1
                    #print(end)
                substring = input_word[start : end + 1]
                substr_list.append(substring)
        #print(substr_list)

        for substr in substr_list:
            max_dim = min(len(substr), 5)
            current_dim = max_dim
            match_temp_array = []

            while current_dim >= 2:
                ngramDict = allGrams[current_dim - 2]

                grams = ngrams(substr, current_dim)
                subsubstr_list = ["".join(g) for g in grams if g.count("_") == 1]
                subsubstr_list = [x for x in subsubstr_list if (x != "_." and x != "._")]
                #print(subsubstr_list)
                
                for subsubstr in subsubstr_list:
                    print(subsubstr)
                    idx = subsubstr.find("_")
                    total_match_count = 0
                    for l in string.ascii_lowercase:
                        filledSubstr = str[0:idx] + l + str[idx+1::]
                        if(ngramDict.get(filledSubstr)):
                            count = ngramDict[filledSubstr]
                            match_temp_array.append([l, filledSubstr, count])
                            total_match_count += count
                            #print(l, count, filledSubstr, sep = " ")
                        for item in match_temp_array:
                            if (item[0] not in guess_list):
                                potential_guesses.append([item[0], current_dim, item[2] / total_match_count, total_match_count])
                
                print(match_temp_array)

                current_dim -= 1
            
        return potential_guesses
        

    def evaluate_potential_guesses(self, potential_guesses):
        #print(potential_guesses)
        if(potential_guesses == []):
            return "!" # no n-gram fits
        # Initialize dictionary to keep scores for each guess
        scores = {}

        # Weighting factors, these are the main thing to tweak...
        weight_probability = self.weight_probability
        weight_length = self.weight_length  # Giving more weight to the length of the n-gram
        weight_count = self.weight_count

        for guess, length, probability, tot_count in potential_guesses:
            guess_letter = list(guess)[0]  # Extract the letter from the set
            #if length == 2:
            #    continue

            # Calculate the score for this guess
            #score = probability * weight_probability + (length * weight_length) + weight_count * np.log(tot_count)
            score = np.log(probability * tot_count) * weight_count + weight_length * length

            # Add or update the score for this letter in the scores dictionary
            if guess_letter in scores:
                scores[guess_letter] += score
            else:
                scores[guess_letter] = score

        # Determine the best guess by finding the maximum score
        #print(scores)
        if len(scores) == 0:
            return "!"
        best_guess = max(scores, key=scores.get)
        return best_guess



    def guess(self, word): # word input example: "_ p p _ e "
        ###############################################
        # Replace with your own "guess" function here #
        ###############################################

        # clean the word so that we strip away the space characters
        # replace "_" with "." as "." indicates any character in regular expressions
        # The extra space characters must be coming from the website...
        clean_word = word.replace(" ", "") #I am putting this here in case I forget later to remove the space stuff when I paste this into the solution
        #clean_word = word.replace("_",".") #I am using underscores

        # find length of passed word
        len_word = len(clean_word)
        
        # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
        current_dictionary = self.current_dictionary
        new_dictionary = []
        #print(current_dictionary[0])
        
        # iterate through all of the words in the old plausible dictionary
        reMatchWord = clean_word.replace("_",".")
        for dict_word in current_dictionary:
            # continue if the word is not of the appropriate length
            if len(dict_word) != len_word:
                continue
                
            # if dictionary word is a possible match then add it to the current dictionary
            if re.match(reMatchWord, dict_word):
                new_dictionary.append(dict_word)
        
        # overwrite old possible words dictionary with updated version
        self.current_dictionary = new_dictionary
        
        
        # count occurrence of all characters in possible word matches
        full_dict_string = "".join(new_dictionary)
        
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()
        #print(sorted_letter_count)                
        
        #guess_letter = '!'

        all_guesses = self.find_potential_guesses(self.allNGrams, word, self.guessed_letters)
        #print(all_guesses)
        ngram_guess = self.evaluate_potential_guesses(all_guesses)
        
        
        # return most frequently occurring letter in all possible words that hasn't been guessed yet
        #print(sorted_letter_count)
        for letter,instance_count in sorted_letter_count:
            if letter not in self.guessed_letters:
                most_common_guess = letter
                #print(most_common_guess)
                break
        
        
        if(ngram_guess != "!"):
            return ngram_guess
        else:
            return most_common_guess
        #return most_common_guess


    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def handle_guess(self, guessLetter, currentWord, ans):
        #I have to convert the strings to lists, find indices of the letter (if present) and then smush the lists back into strings and spit it out
        indices = [i for i, x in enumerate(list(ans)) if x == guessLetter]
        wordList = list(currentWord)
        for i in indices:
            wordList[i] = guessLetter #now I replace the occurrences of the letter
        word = "".join(wordList)
        return word

    def get_winstate(self):
        return self.has_won
    
    def get_gamelog(self):
        return self.gameLog
        
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.gameLog = []
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
        self.has_won = False
        #pick a word from the dictionary at random.
        #correctWord = random.choice(self.full_dictionary) #this is the initial word, but need to update it based on guesses
        self.generate_correct_word()
        word = "_" * len(self.correct_word) #I should make this a member of self right?

        tries_remains = 6 #I think this needs to just be a magic number?
        
        self.gameLog.append([self.correct_word, word, self.guessed_letters.copy(), tries_remains])
        if verbose:
            print("Successfully start a new game! # of tries remaining: {0}. Word: {1}.".format(tries_remains, word))
        while tries_remains >0:
            #tries_remains -=1 tries remains decreases after WRONG answer

            #get guessed letter
            guess_letter = self.guess(word)

            #append guessed letter to guessed letters field in hangman object
            self.guessed_letters.append(guess_letter)
            

            if verbose:
                print("Guessing letter: {0}".format(guess_letter))

            #apply guessed letter to the word
            new_word = self.handle_guess(guess_letter, word, self.correct_word) 

            #check if we have the word
            if new_word == self.correct_word:
                word = new_word
                if verbose:
                    print("Success! the word was: %s" % word)
                self.has_won = True
                self.gameLog.append([self.correct_word, word, self.guessed_letters.copy(), tries_remains])
                break
            
            if new_word == word:
                #this means that we did not get a correct guess
                #decrease number of tries
                if verbose:
                    print("Incorrect guess {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))
                tries_remains -=1
            else:
                #We have a correct letter guessed, but not the complete word
                #don't decrement tries_remains
                word = new_word
                if verbose:
                    print("Got a Letter, {0}, # of tries remaining: {1}. Word: {2}.".format(guess_letter,tries_remains, word))

            self.gameLog.append([self.correct_word, word, self.guessed_letters.copy(), tries_remains])
        if tries_remains == 0:
            if verbose:
                print("You Lose, the answer was: %s" % self.correct_word)
                



So... implementing a "probability" actually decreased how good the model is which is kinda sad...

Also, even after messing with the code, it should be the same result as in 6 but it's not...

So gotta fix that...

maybe the issue is the 2-grams... idk maybe it's better to start at 3...

In [45]:
game = MyHangman(weight_count = 0.25, weight_length = 2)
game.set_correct_word("ptilo")

In [46]:
game.start_game()
print()

Successfully start a new game! # of tries remaining: 6. Word: _____.
[]
[]
Guessing letter: a
Incorrect guess a, # of tries remaining: 6. Word: _____.
[]
[]
Guessing letter: e
Incorrect guess e, # of tries remaining: 5. Word: _____.
[]
[]
Guessing letter: s
Incorrect guess s, # of tries remaining: 4. Word: _____.
[]
[]
Guessing letter: o
Got a Letter, o, # of tries remaining: 3. Word: ____o.
[]
_o.
ae
be
ce
de
ee
fe
ge
he
ie
je
ke
le
me
ne
oe
pe
qe
re
se
te
ue
ve
we
xe
ye
ze
[]
_o
ae
ae
be
be
ce
ce
de
de
ee
ee
fe
fe
ge
ge
he
he
ie
ie
je
je
ke
ke
le
le
me
me
ne
ne
oe
oe
pe
pe
qe
qe
re
re
se
se
te
te
ue
ue
ve
ve
we
we
xe
xe
ye
ye
ze
ze
[['a', 'ae', 3350], ['b', 'be', 5974], ['c', 'ce', 9023], ['d', 'de', 13954], ['e', 'ee', 5426], ['f', 'fe', 3328], ['g', 'ge', 7326], ['h', 'he', 11707], ['i', 'ie', 8301], ['j', 'je', 735], ['k', 'ke', 5188], ['l', 'le', 22135], ['m', 'me', 11077], ['n', 'ne', 19584], ['o', 'oe', 1715], ['p', 'pe', 10209], ['q', 'qe', 5], ['r', 're', 24800], ['s', 'se', 

In [25]:
game.changeWeights(1,1,1)

Ok... I have successfully replicated how NGRAM 6 works... so now I can see whether that method is indeed better... In the previous file I tweaked way too much stuff. I modified how the ngrams are stored and accessed, from freq_dist in the nltk package, to dictionaries. This was to speed up the actual calculations. But I also modified how the algorithm picked guesses...

So let's go back to tweaking parameters?

In [ ]:
game.get_winstate()

True

dict reducing method is ~87% on TRAINING data but bad on new data

In [28]:
game = MyHangman(weight_count = 0.25, weight_length = 2)
wincount = 0
N = 1000
for i in range(N):
    if i % (N/10) == 0:
        print("on game %i of %i" % (i+1, N))
        win_percent = 100*wincount/(i+1)
        print("win percentage: %f" % win_percent)
    game.start_game(verbose=False)
    wincount += game.get_winstate()
print()
print("win percentage: ", 100*wincount/N, "%")

on game 1 of 1000
win percentage: 0.000000
on game 101 of 1000
win percentage: 55.445545
on game 201 of 1000
win percentage: 56.218905
on game 301 of 1000
win percentage: 54.152824
on game 401 of 1000
win percentage: 55.112219
on game 501 of 1000
win percentage: 57.285429
on game 601 of 1000
win percentage: 57.237937
on game 701 of 1000
win percentage: 58.059914
on game 801 of 1000
win percentage: 58.926342
on game 901 of 1000
win percentage: 58.712542

win percentage:  59.6 %


Yeah, so picking the longest substring to make a guess gives ~10% gain in accuracy... cool. At least with the current parameters. Tweaking them could improve performance.

In [30]:
game = MyHangman()
gridScores = []
weight_count_vector = np.linspace(0, 0.5, 11)
weight_length_vector = np.linspace(0, 5, 11)
M = len(weight_count_vector) * len(weight_length_vector)
#MyHangman(weight_count = 0.25, weight_length = 2) best performance so far...

tr = 0
for wc in weight_count_vector:
    for wl in weight_length_vector:
        tr += 1
        game.changeWeights(wl, 0, wc)
        wincount = 0
        N = 1000
        for i in range(N):
            #if i % (N/10) == 0:
                #print("on game %i of %i" % (i+1, N))
                #win_percent = 100*wincount/(i+1)
                #print("win percentage: %f" % win_percent)
            game.start_game(verbose=False)
            wincount += game.get_winstate()
        print("Finished trial %i of %i" % (tr, M))
        print("win percentage: ", 100*wincount/N, "%")
        print()
        gridScores.append([wl, wc, 100*wincount/N])

Finished trial 1 of 121
win percentage:  4.8 %

Finished trial 2 of 121
win percentage:  14.2 %

Finished trial 3 of 121
win percentage:  17.8 %

Finished trial 4 of 121
win percentage:  14.6 %

Finished trial 5 of 121
win percentage:  14.8 %

Finished trial 6 of 121
win percentage:  16.5 %

Finished trial 7 of 121
win percentage:  18.3 %

Finished trial 8 of 121
win percentage:  17.8 %

Finished trial 9 of 121
win percentage:  16.5 %

Finished trial 10 of 121
win percentage:  16.3 %

Finished trial 11 of 121
win percentage:  17.7 %

Finished trial 12 of 121
win percentage:  60.4 %

Finished trial 13 of 121
win percentage:  61.0 %

Finished trial 14 of 121
win percentage:  58.3 %

Finished trial 15 of 121
win percentage:  61.5 %

Finished trial 16 of 121
win percentage:  60.3 %

Finished trial 17 of 121
win percentage:  63.3 %

Finished trial 18 of 121
win percentage:  61.0 %

Finished trial 19 of 121
win percentage:  60.7 %

Finished trial 20 of 121
win percentage:  60.8 %

Finished t

In [31]:
gridScores.sort(key = lambda x: x[2], reverse = True)
top_20 = gridScores[0:20]
print(top_20)

[[3.0, 0.45, 64.9], [0.5, 0.1, 64.1], [2.5, 0.45, 64.1], [3.5, 0.35000000000000003, 64.0], [1.0, 0.45, 64.0], [0.5, 0.15000000000000002, 63.9], [3.5, 0.2, 63.6], [3.0, 0.30000000000000004, 63.5], [0.5, 0.5, 63.4], [2.0, 0.5, 63.4], [2.5, 0.05, 63.3], [2.0, 0.2, 63.3], [4.5, 0.4, 63.2], [4.5, 0.45, 63.2], [2.5, 0.15000000000000002, 63.1], [4.5, 0.05, 63.0], [3.5, 0.4, 63.0], [3.0, 0.1, 62.9], [5.0, 0.5, 62.9], [0.5, 0.4, 62.8]]


In [4]:
top_20 = [[3.0, 0.45, 64.9], [0.5, 0.1, 64.1], [2.5, 0.45, 64.1], [3.5, 0.35000000000000003, 64.0], [1.0, 0.45, 64.0], [0.5, 0.15000000000000002, 63.9], [3.5, 0.2, 63.6], [3.0, 0.30000000000000004, 63.5], [0.5, 0.5, 63.4], [2.0, 0.5, 63.4], [2.5, 0.05, 63.3], [2.0, 0.2, 63.3], [4.5, 0.4, 63.2], [4.5, 0.45, 63.2], [2.5, 0.15000000000000002, 63.1], [4.5, 0.05, 63.0], [3.5, 0.4, 63.0], [3.0, 0.1, 62.9], [5.0, 0.5, 62.9], [0.5, 0.4, 62.8]]

Ok, so I have the 20 best set, let's check them with a bit more trials

In [7]:
game = MyHangman()

In [8]:
M = len(top_20)
#MyHangman(weight_count = 0.25, weight_length = 2) best performance so far...

tr = 0
top_20_checked = []
for item in top_20:
    wl = item[0]
    wc = item[1]
    tr += 1
    game.changeWeights(wl, 0, wc)
    wincount = 0
    N = 5000
    for i in range(N):
        if i % (N/10) == 0:
            print("on game %i of %i" % (i+1, N))
            win_percent = 100*wincount/(i+1)
            print("win percentage: %f" % win_percent)
        game.start_game(verbose=False)
        wincount += game.get_winstate()
    print("Finished trial %i of %i" % (tr, M))
    print("win percentage: ", 100*wincount/N, "%")
    print()
    top_20_checked.append([wl, wc, 100*wincount/N])

on game 1 of 5000
win percentage: 0.000000
on game 501 of 5000
win percentage: 56.287425
on game 1001 of 5000
win percentage: 58.641359
on game 1501 of 5000
win percentage: 59.626915
on game 2001 of 5000
win percentage: 60.719640
on game 2501 of 5000
win percentage: 60.255898
on game 3001 of 5000
win percentage: 60.479840
on game 3501 of 5000
win percentage: 60.582691
on game 4001 of 5000
win percentage: 60.609848
on game 4501 of 5000
win percentage: 61.141968
Finished trial 1 of 20
win percentage:  60.94 %

on game 1 of 5000
win percentage: 0.000000
on game 501 of 5000
win percentage: 63.073852
on game 1001 of 5000
win percentage: 59.840160
on game 1501 of 5000
win percentage: 60.293138
on game 2001 of 5000
win percentage: 60.319840
on game 2501 of 5000
win percentage: 61.255498
on game 3001 of 5000
win percentage: 61.146285
on game 3501 of 5000
win percentage: 61.153956
on game 4001 of 5000
win percentage: 60.859785
on game 4501 of 5000
win percentage: 61.364141
Finished trial 2 of 2

In [9]:
top_20_checked.sort(key = lambda x: x[2], reverse = True)
print(top_20_checked)

[[0.5, 0.5, 62.7], [2.0, 0.2, 62.5], [0.5, 0.4, 62.1], [1.0, 0.45, 61.98], [3.5, 0.2, 61.84], [4.5, 0.4, 61.84], [2.5, 0.45, 61.82], [0.5, 0.1, 61.54], [5.0, 0.5, 61.54], [0.5, 0.15000000000000002, 61.52], [3.5, 0.4, 61.52], [2.5, 0.05, 61.28], [3.0, 0.1, 61.28], [3.0, 0.30000000000000004, 61.2], [3.0, 0.45, 60.94], [4.5, 0.45, 60.88], [4.5, 0.05, 60.88], [3.5, 0.35000000000000003, 60.46], [2.5, 0.15000000000000002, 60.14], [2.0, 0.5, 60.1]]


In [ ]:
top_20_checked = [[0.5, 0.5, 62.7], [2.0, 0.2, 62.5], [0.5, 0.4, 62.1], [1.0, 0.45, 61.98], [3.5, 0.2, 61.84], [4.5, 0.4, 61.84], [2.5, 0.45, 61.82], [0.5, 0.1, 61.54], [5.0, 0.5, 61.54], [0.5, 0.15000000000000002, 61.52], [3.5, 0.4, 61.52], [2.5, 0.05, 61.28], [3.0, 0.1, 61.28], [3.0, 0.30000000000000004, 61.2], [3.0, 0.45, 60.94], [4.5, 0.45, 60.88], [4.5, 0.05, 60.88], [3.5, 0.35000000000000003, 60.46], [2.5, 0.15000000000000002, 60.14], [2.0, 0.5, 60.1]][[0.5, 0.5, 62.7], [2.0, 0.2, 62.5], [0.5, 0.4, 62.1], [1.0, 0.45, 61.98], [3.5, 0.2, 61.84], [4.5, 0.4, 61.84], [2.5, 0.45, 61.82], [0.5, 0.1, 61.54], [5.0, 0.5, 61.54], [0.5, 0.15000000000000002, 61.52], [3.5, 0.4, 61.52], [2.5, 0.05, 61.28], [3.0, 0.1, 61.28], [3.0, 0.30000000000000004, 61.2], [3.0, 0.45, 60.94], [4.5, 0.45, 60.88], [4.5, 0.05, 60.88], [3.5, 0.35000000000000003, 60.46], [2.5, 0.15000000000000002, 60.14], [2.0, 0.5, 60.1]]

In [27]:
print("win percentage: ", 100*wincount/N, "%")

win percentage:  64.0 %


This is a significant improvement... hopefully... but it still isn't good enough in my opinion... I at least have parameters to tweak? It would be easier to mass tweak parameters if it was faster.

Am I doing things wrong? I am still using the FULL dictionary to train the algorithm and test it. I need to test the algorithm on UNKNOWN words. I need to do a test/train split or load another data file.

In [28]:
tf = open("words_250000_train.txt","r")
full_dict = tf.read().splitlines()
tf.close()

In [29]:
tf = open("words_test.txt","r")
test_dict = tf.read().splitlines()
tf.close()

In [30]:
len(full_dict)

227300

In [31]:
len(test_dict)

170671

In [32]:
len(full_dict)/len(test_dict)

1.3318021222117407

In [21]:
def generateNGrams(vocab, n):
    #return the ngrams as dictionaries
    #this is a little confusing because I am loading words from a dictionary (not the data type) of words...
    #let's call it vocab instead
    ngrams = {}
    for word in vocab:
        word = "." + word + "." # pad with start and end bits
        if len(word) < n:
            continue
        for i in range(len(word) - n + 1):
            gram = word[i:i+n]
            if gram not in ngrams:
                ngrams[gram] = 1
            else:
                ngrams[gram] += 1
    return ngrams

ngrams=[generateNGrams(full_dict,i) for i in range(1,6)]


In [7]:

def getNGrams(words,n):
    ngrams = {}
    for word in words:
        if len(word) <n:
            continue
        for i in range(len(word)-n+1):
            ngram = word[i:i+n]
            if ngram not in ngrams:
                ngrams[ngram] = 1
            else:
                ngrams[ngram]+=1
    return ngrams

ngrams=[getNGrams(full_dict,i) for i in range(1,6)]

In [16]:

def ngramprob(word,letter):
        denominator=0
        underscroll_location=word.find("_")
        for l in string.ascii_lowercase:            
            try:
                denominator+=ngrams[len(word)-1][word[:underscroll_location]+l+word[underscroll_location+1:]]
                print(word[:underscroll_location]+l+word[underscroll_location+1:])
            except:
                pass
        return ngrams[len(word)-1][word[:underscroll_location]+letter+word[underscroll_location+1:]]/denominator

In [19]:
ngramprob("pp_e","l")

ppae
ppee
pphe
ppie
pple
ppne
ppre


0.38485804416403785

In [13]:
ngrams

[{'a': 179837,
  's': 148462,
  'c': 89367,
  'h': 58051,
  'e': 233745,
  'n': 152259,
  'g': 51850,
  'd': 74856,
  'l': 122431,
  'u': 77304,
  'i': 184746,
  't': 137277,
  'm': 62191,
  'o': 150052,
  'p': 65785,
  'r': 149228,
  'v': 21057,
  'k': 18685,
  'w': 17732,
  'f': 26431,
  'b': 39840,
  'x': 6050,
  'y': 40985,
  'j': 3790,
  'z': 8749,
  'q': 3986},
 {'aa': 270,
  'as': 10554,
  'ac': 10476,
  'ch': 12023,
  'he': 11707,
  'en': 23199,
  'ae': 3350,
  'ee': 5426,
  'ag': 4914,
  'ah': 1089,
  'ed': 23098,
  'hs': 600,
  'al': 23550,
  'le': 22135,
  'es': 27128,
  'su': 5905,
  'un': 15154,
  'nd': 9759,
  'li': 19202,
  'ii': 421,
  'is': 20485,
  'ls': 2019,
  'st': 20183,
  'am': 6969,
  'an': 25117,
  'da': 5429,
  'hl': 1069,
  'ao': 256,
  'ap': 6038,
  'ps': 2219,
  'ss': 12161,
  'ar': 19848,
  'ra': 20481,
  'au': 3471,
  'rd': 3773,
  'dv': 270,
  'va': 3060,
  'rk': 1282,
  'dw': 387,
  'wo': 2508,
  'ol': 10801,
  'lf': 1849,
  're': 24800,
  'rg': 2254,
 

In [32]:
ngrams[2]["xyz"]

KeyError: 'xyz'

It is a bit absurd how much faster this code is...

In [15]:
ngrams[2]

{'aaa': 8,
 'aas': 18,
 'aac': 6,
 'ach': 1469,
 'che': 2342,
 'hen': 898,
 'aae': 3,
 'aee': 9,
 'aag': 9,
 'aah': 4,
 'ahe': 115,
 'hed': 967,
 'ahs': 78,
 'aal': 42,
 'ale': 1452,
 'les': 2703,
 'esu': 262,
 'sun': 237,
 'und': 2436,
 'ali': 3714,
 'lii': 33,
 'iis': 18,
 'als': 470,
 'lst': 141,
 'aam': 26,
 'aan': 29,
 'and': 2651,
 'nda': 774,
 'dah': 59,
 'ahl': 35,
 'aao': 1,
 'aap': 5,
 'aps': 262,
 'pss': 5,
 'aar': 39,
 'ara': 1633,
 'rau': 209,
 'ard': 1887,
 'rdv': 14,
 'dva': 52,
 'var': 331,
 'ark': 509,
 'rdw': 36,
 'dwo': 86,
 'wol': 100,
 'olf': 90,
 'are': 1221,
 'ren': 1379,
 'arg': 568,
 'rgh': 47,
 'ari': 3006,
 'rik': 107,
 'ika': 101,
 'aro': 663,
 'ron': 1649,
 'oni': 3077,
 'nic': 1924,
 'nit': 1372,
 'ite': 2985,
 'ons': 2574,
 'nsb': 41,
 'sbu': 153,
 'bur': 766,
 'urg': 527,
 'arp': 439,
 'arr': 968,
 'rrg': 3,
 'ghh': 28,
 'asv': 7,
 'svo': 5,
 'vog': 22,
 'oge': 888,
 'gel': 399,
 'els': 362,
 'aau': 7,
 'aup': 59,
 'aav': 1,
 'avs': 8,
 'vso': 2,
 'aba':

In [51]:
temp = {"a" : 1, 
        "b" : 2, 
        "c" : 3}

In [58]:
for l in string.ascii_lowercase:
    if(temp.get(l)):
        print("%s in dict" % l)
    print(temp.get(l))

a in dict
1
b in dict
2
c in dict
3
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [54]:
temp.get("z")

In [ ]:
thisdict =	{
  "brand": "Ford",
  "model": "Mustang",
  "year": 1964
}

In [50]:
thisdict["brand"]

'Ford'

In [46]:
ngrams[2][".aa"]

32

In [165]:
test_word = "abc_efgh"
idx = test_word.index("_")
max_dim = len(test_word)



In [174]:
substr_lists = []
input_word = "abc_efgh"
input_word = "." + input_word + "." #pad beginning and end with something to indicate beginning/end
#word = temp_word
#print(input_word)
for i,char in enumerate(input_word):
    if char == "_": #found a blank
        start, end = i, i
        #print(i)
        #get longest substring containing the blank
        while start > 0 and input_word[start-1] != "_":
            start-=1
            #print(start)
        while end < len(input_word) - 1 and input_word[end+1] != "_":
            end += 1
            #print(end)
        #now, turn substring into something to match an n-gram
        substring = input_word[start : end + 1]
        #Ok, this is the LONGEST SUBSTRING CONTAINING ONE UNDERSCORE
        #We now have to break it into grams and add them to the substr_lists

        #I tested the algorithm and this doesn't perform all that well, so instead of only considering the longest substring I will consider ALL substrings and weight them by length...
        for j in range(2, min(6, len(substring)+1)):
            grams = ngrams(substring, j)
            for g in grams:
                if(g.count("_") == 1 and set(ssList) - set({None, "_"}) != set()):
                    substr_lists.append("".join(g))
print(substr_lists)

['c_', '_e', 'bc_', 'c_e', '_ef', 'abc_', 'bc_e', 'c_ef', '_efg', '.abc_', 'abc_e', 'bc_ef', 'c_efg', '_efgh']


In [171]:
for g in ngrams(input_word, 3):
    print(g)

('.', 'a', 'b')
('a', 'b', 'c')
('b', 'c', '_')
('c', '_', 'e')
('_', 'e', 'f')
('e', 'f', 'g')
('f', 'g', 'h')
('g', 'h', '.')


In [172]:
temp = ['.abc_', 'abc_e', 'bc_ef', 'c_efg', '_efgh']

In [224]:
set("._") - set({".", "_"})

set()

In [ ]:
substr = "abc_efg"
#print(substr)
#print("substring: " + substr)
#substr = string(substr)
max_dimension = len(substr)

#start at max_dim, and work way down to 2
#only consider the longest matches.
dimension = max_dimension
while dimension >= 2:
    #ngramDict = allGrams[dimension-2]

    match_temp_array = []
    idx = substr.find("_") # get the index of the blank in this substring
    #guessableLetters = set(self.alphabet) - set(guess_list)
    
    total_match_count = 0
    for l in string.ascii_lowercase:
        filledSubstr = substr[0:idx] + l + substr[idx+1::]
        
        if(ngramDict.get(filledSubstr)):
            count = ngramDict[filledSubstr]
            match_temp_array.append([l, filledSubstr, count])
            total_match_count += count
            #print(l, count, filledSubstr, sep = " ")
    
    for item in match_temp_array:
        if (item[0] not in guess_list):
            potential_guesses.append([item[0], dimension, item[2] / total_match_count, total_match_count])
    
    if match_temp_array != []:
        dimension -= 1

In [12]:
substr = "abc_efg"
max_dim = len(substr)
idx = substr.find("_")

dimension = max_dim
while dimension > 1:
    grams = ngrams(substr, dimension)
    substr_grams = ["".join(g) for g in grams if g.count("_") == 1]

    for str in substr_grams:
        print(str)

    



    dimension -= 1

abc_efg
abc_ef
bc_efg
abc_e
bc_ef
c_efg
abc_
bc_e
c_ef
_efg
bc_
c_e
_ef
c_
_e


In [13]:
substr = "c_ef"
max_dim = len(substr)
idx = substr.find("_")

dimension = max_dim
while dimension > 1:
    grams = ngrams(substr, dimension)
    substr_grams = ["".join(g) for g in grams if g.count("_") == 1]

    for str in substr_grams:
        print(str)

    



    dimension -= 1

c_ef
c_e
_ef
c_
_e


In [17]:
input_word = "abc_efgh_j"

#Ok, so we need to get the longest substrings in the word that contain ONE blank space.
#first pad beginning and end of word with something, will replace with None later
substr_list = []
input_word = "." + input_word + "." #pad beginning and end with something to indicate beginning/end

for i,char in enumerate(input_word):
    if char == "_": #found a blank
        start, end = i, i
        #print(i)
        #get longest substring containing the blank
        while start > 0 and input_word[start-1] != "_":
            start-=1
            #print(start)
        while end < len(input_word) - 1 and input_word[end+1] != "_":
            end += 1
            #print(end)
        substring = input_word[start : end + 1]
        substr_list.append(substring)
#print(substr_list)

potential_matches = []
for substr in substr_list:
    max_dim = min(len(substr), 5)
    current_dim = max_dim

    while current_dim >= 2:

        grams = ngrams(substr, current_dim)
        subsubstr_list = ["".join(g) for g in grams if g.count("_") == 1]
        #print(subsubstr_list)
        
        for subsubstr in subsubstr_list:
            idx = subsubstr.find("_")
            total_match_count = 0

        #now match stuff here...
        
        current_dim -= 1


['.abc_efgh', 'efgh_j.']


['.abc_', 'abc_e', 'bc_ef', 'c_efg', '_efgh']
['abc_', 'bc_e', 'c_ef', '_efg']
['bc_', 'c_e', '_ef']
['c_', '_e']
['efgh_', 'fgh_j', 'gh_j.']
['fgh_', 'gh_j', 'h_j.']
['gh_', 'h_j', '_j.']
['h_', '_j']


In [ ]:
input_word = "abc_efgh_j"

potential_guesses = []
remove_set = set(guess_list)
remove_set.add(None)

#Ok, so we need to get the longest substrings in the word that contain ONE blank space.
#first pad beginning and end of word with something, will replace with None later
substr_lists = []
input_word = "." + input_word + "." #pad beginning and end with something to indicate beginning/end

for i,char in enumerate(input_word):
    if char == "_": #found a blank
        start, end = i, i
        #print(i)
        #get longest substring containing the blank
        while start > 0 and input_word[start-1] != "_":
            start-=1
            #print(start)
        while end < len(input_word) - 1 and input_word[end+1] != "_":
            end += 1
            #print(end)
        substring = input_word[start : end + 1]
        #Ok, this is the LONGEST SUBSTRING CONTAINING ONE UNDERSCORE
        #We now have to break it into grams and add them to the substr_lists

        #I tested the algorithm and this doesn't perform all that well, so instead of only considering the longest substring I will consider ALL substrings and weight them by length...
        #for j in range(2, min(6, len(substring)+1)):
        #    grams = ngrams(substring, j)
        #    for g in grams:
        #        if(g.count("_") == 1 and set(g) - set({".", "_"}) != set()):
        #            substr_lists.append("".join(g))

        #I tested the algorithm AGAIN and it performs WORSE when you consider smaller substrings?
        if len(substring) > 5:
            grams = ngrams(substring, 5)
            for g in grams:
                if(g.count("_") == 1 and set(g) - set({".", "_"}) != set()):
                    substr_lists.append("".join(g))
        else:
            if(substring.count("_") == 1 and set(substring) - set({".", "_"}) != set()):
                    substr_lists.append(substring)
#Ok, now all substrings are in the ssList
#the code is also only letting the longest substring match...
#Ok... so the code can't handle if there is NO match in the set of ngrams, if the substring length is 5, it only checks 5grams for example...
            
#For each substring, need to get the prob of each match, and I am doing this by dividing the prob of each match by the prob of all matches for a given substring.
for substr in substr_lists:
    #print(substr)
    #print("substring: " + substr)
    #substr = string(substr)
    max_dimension = len(substr)

    #start at max_dim, and work way down to 2
    #only consider the longest matches.
    max_dim = len(substr)
    idx = substr.find("_")
    total_match_count = 0

    dimension = max_dim

    #I am doing some repeat work but I modified the code again.
    #substr_lists contains the longest substrings of our word that contain only one blank
    #unless the longest substring is longer than 5 in which case it is broken down into 5-grams.
    #Now, when we are picking words from the dictionary, if we break a word into an n-gram of any size, we are guaranteed to find a match
    #That isn't the case if we pick words from OUTSIDE the dictionary, like on a test set.
    #So, if the substr is length 5, we have to look at SHOOT THIS IS ALL WRONG!!!

    while dimension > 1:
        match_temp_array = []
        grams = ngrams(substr, dimension)
        substr_grams = ["".join(g) for g in grams if g.count("_") == 1]
        ngramDict = allGrams[dimension-2]

        for str in substr_grams:
            for l in string.ascii_lowercase:
                filledSubstr = str[0:idx] + l + str[idx+1::]
                
                if(ngramDict.get(filledSubstr)):
                    count = ngramDict[filledSubstr]
                    match_temp_array.append([l, filledSubstr, count])
                    total_match_count += count
                    #print(l, count, filledSubstr, sep = " ")
        
        for item in match_temp_array:
            if (item[0] not in guess_list):
                potential_guesses.append([item[0], dimension, item[2] / total_match_count, total_match_count])
    
        if len(match_temp_array) > 0:
            break
        

        dimension -= 1







    dimension = max_dimension
    while dimension >= 2:
        ngramDict = allGrams[dimension-2]

        match_temp_array = []
        idx = substr.find("_") # get the index of the blank in this substring
        #guessableLetters = set(self.alphabet) - set(guess_list)
        total_match_count = 0
        for l in string.ascii_lowercase:
            filledSubstr = substr[0:idx] + l + substr[idx+1::]
            
            if(ngramDict.get(filledSubstr)):
                count = ngramDict[filledSubstr]
                match_temp_array.append([l, filledSubstr, count])
                total_match_count += count
                #print(l, count, filledSubstr, sep = " ")
        
        for item in match_temp_array:
            if (item[0] not in guess_list):
                potential_guesses.append([item[0], dimension, item[2] / total_match_count, total_match_count])
        
        if match_temp_array != []:
            dimension -= 1
    
return potential_guesses